# 5. Step-Back Prompting

**What:** Generate a broader question first to get background context

**Why:** Combines general principles with specific details

**When:** Technical/conceptual questions that need background

**Key Idea:** "Zoom out before zooming in"

**Flow:**
```
Specific Question → General Step-Back Question
        ↓                      ↓
  Specific Docs          General Docs
        ↓                      ↓
         Combine → Final Answer
```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from config import model, setup_vectorstore, get_retriever, format_docs

## Generate Step-Back Question

In [ ]:
def generate_step_back(question):
    template = """Generate a broader, more general version of this question.
Ask about principles, concepts, or background knowledge.

Specific: {question}

General (ONE question only):"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"question": question}).strip()

# Test
q = "What optimizer did the RL model use?"
print(f"Specific: {q}")
print(f"Step-back: {generate_step_back(q)}")

## Step-Back Retrieval

In [ ]:
def step_back_retrieve(question, retriever, k=3):
    print(f"Specific: {question}\n")
    
    step_back_q = generate_step_back(question)
    print(f"Step-Back: {step_back_q}\n")
    
    general_docs = retriever.invoke(step_back_q)[:k]
    specific_docs = retriever.invoke(question)[:k]
    
    print(f"Retrieved: {len(general_docs)} general + {len(specific_docs)} specific")
    
    return general_docs, specific_docs, step_back_q

## Complete Step-Back RAG

In [ ]:
def step_back_rag(question, retriever):
    general_docs, specific_docs, step_back_q = step_back_retrieve(question, retriever)
    
    general_context = format_docs(general_docs)
    specific_context = format_docs(specific_docs)
    
    template = """Answer using both general principles and specific details.

GENERAL (principles):
{general_context}

SPECIFIC (details):
{specific_context}

Question: {question}

Answer:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    answer = chain.invoke({
        "general_context": general_context,
        "specific_context": specific_context,
        "question": question
    })
    
    print(f"\nAnswer: {answer}")
    return answer

## Test

In [ ]:
vectorstore = setup_vectorstore()
retriever = get_retriever(vectorstore, k=5)

test_questions = [
    "What role does DeMask play?",
    "What is the difference between Graph DTA and Graph DF?"
]

for q in test_questions:
    print("="*60)
    step_back_rag(q, retriever)
    print()